# Simulating Noise On Amazon Braket

In this hands-on lab, we show you how you can simulate the effect of noise on your circuits.

This notebook in intended to help you get started. Additional learning material can be found in the [Amazon Braket Examples GitHub repository](https://github.com/aws/amazon-braket-examples/tree/main): 
* `Braket examples/braket_features/Simulating_Noise_On_Amazon_Braket.ipynb` provides a more detailed overview of noise simulations on Amazon Braket
* `Braket examples/braket_features/Noise_models/Noise_models_on_Amazon_Braket.ipynb` introduces noise models on Amazon Braket
* `Braket examples/braket_features/Noise_models/Noise_models_on_Rigetti.ipynb` shows you how to construct a noise model from device calibration data for real quantum processing units

In [ ]:
# general imports
import string
import time
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Braket SDK module imports
from braket.circuits import Circuit, Gate, Instruction, Observable
from braket.devices import LocalSimulator, Devices
from braket.aws import AwsDevice, AwsQuantumTask

## Build an Example Circuit

First, we build a [GHZ](https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state) circuit with a parametrized subroutine.


In [ ]:
# function to build a GHZ state
def ghz(n_qubits):
    """
    function to return a GHZ circuit ansatz
    input: number of qubits
    """

    # instantiate circuit object
    circuit = Circuit()

    # add Hadamard gate on first qubit
    circuit.h(0)

    # apply series of CNOT gates
    for ii in range(0, n_qubits-1):
        circuit.cnot(control=ii, target=ii+1)

    return circuit


# register the subroutine
Circuit.register_subroutine(ghz)

Let's create an instance of a 3-qubit GHZ circuit.

In [ ]:
# create the circuit
n_qubits = 3
ghz = Circuit().ghz(n_qubits)

print(ghz)

## Simulate the Circuit without Noise

First, we review the results we expect for executing the circuit in a noise-free environment.

In [ ]:
# set up device: Local Simulator
device = LocalSimulator()

In [ ]:
# run circuit
result = device.run(ghz, shots=1000).result()
# get measurement shots
counts = result.measurement_counts
# print counts
print(counts)

In [ ]:
# plot using Counter
plt.bar(counts.keys(), counts.values())
plt.xlabel('bitstrings')
plt.ylabel('counts')
plt.draw()

## Execute the Circuit on a real QPU

Now, let's run the same circuit on a real QPU and compare the results with the above simulation.

In [ ]:
rigetti = AwsDevice(Devices.Rigetti.AspenM3)

In [ ]:
rigetti_task = rigetti.run(ghz, shots=1000)
print(rigetti_task)

In [ ]:
# check status
print('Status of task:', rigetti_task.state())

In [ ]:
# retrieve task
task = rigetti_task
#task = AwsQuantumTask(arn='')

# print status
status = task.state()
print('Status of (reconstructed) task:', status)
print('\n')
# wait for job to complete
# terminal_states = ['COMPLETED', 'FAILED', 'CANCELLED']
if status == 'COMPLETED':
    # get results
    results = task.result()

    # get all metadata of submitted task
    metadata = task.metadata()
    # example for metadata
    shots = metadata['shots']
    machine = metadata['deviceArn']
    # print example metadata
    print("{} shots taken on machine {}.\n".format(shots, machine))

    # get the compiled circuit
    print("The compiled circuit is:\n", results.additional_metadata.rigettiMetadata.compiledProgram)

    # get measurement counts
    counts = results.measurement_counts
    print('Measurement counts:', counts)

    # plot results: see effects of noise
    plt.bar(counts.keys(), counts.values())
    plt.xlabel('bitstrings')
    plt.ylabel('counts')
    plt.tight_layout()
    plt.draw()
elif status in ['FAILED', 'CANCELLED']:
    # print terminal message
    print('Your task is in terminal status, but has not completed.')
else:
    # print current status
    print('Sorry, your task is still being processed and has not been finalized yet.')

## Add Noise to our simulation

Building on the knowledge you already have, create another GHZ circuit with 3 qubits - but this time, let's add a noise channel to it.

For that purpose, we need to import the Noise library from Braket SDK.

In [ ]:
from braket.circuits import Noise

In [ ]:
# define a GHZ circuit using previous function
n_qubits = 3
ghz_with_noise = Circuit().ghz(n_qubits)

To add bit flip noise with a 10% of probabilty to happen in all gate operations in the circuit, we need to call the `Noise` object with the `BitFlip` method and the correct parameter. Then we can apply the noise channel to the circuit by calling the method `apply_gate_noise`:

In [ ]:
# define a noise channel
noise = Noise.BitFlip(probability=0.10)

# add noise to every gate in the circuit
ghz_with_noise.apply_gate_noise(noise)

print(ghz)

Now, we need to instantiate a noise simulator. We can choose between the local simulator and the DM1 on-demand simulator.

In [ ]:
# select the Local Simulator and pass the noise parameter "braket_dm"
device = LocalSimulator("braket_dm")

# select the on-demand density matrix simulator DM1
#device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")

Now we just need to execute the circuit like any other (noise-free) circuit, and plot your results:

In [ ]:
# run the circuit
task = device.run(ghz_with_noise, shots=1000)

# visualize the results
result = task.result()
measurement = result.measurement_counts
print('measurement results:', measurement)

In [ ]:
plt.bar(measurement.keys(), measurement.values())
plt.xlabel('bitstrings')
plt.ylabel('counts')
plt.tight_layout()
plt.draw()

<div class="alert alert-block alert-info">
    How do the results from the QPU and DM1 compare? Did you get similar results? If not, why do you think that could be? </br>
    <b>Taking it further: </b> Explore more noise channels. Change the previous circuit to add a depolarizing noise channel at 0.05 probability. <a href="https://docs.aws.amazon.com/braket/latest/developerguide/braket-constructing-circuit.html#noise-simulation"> More examples here. </a> 
</div>

## Other Ways to Add Noise Channels to your Circuits

Instead of adding a noise channel globally to all logic operations in your circuit, the SDK provides options to model noise more fine-grained.

Let's start again creating a 5-qubit GHZ circuit instance.

In [ ]:
my_circuit = Circuit().ghz(5)
print(my_circuit)

Now, let's add the bit flip noise channel only to the CNOT gates in our circuit:

In [ ]:
my_circuit.apply_gate_noise(Noise.BitFlip(probability=0.1), target_gates=[Gate.CNot])
print(my_circuit)

In addition, let's assume we want to had a depolarizing noise channel to the gates acting on the qubits `q0` and `q2`:

In [ ]:
my_circuit.apply_gate_noise(Noise.Depolarizing(probability=0.1), target_qubits=[0, 2])
print(my_circuit)

Also, we can simulate noise becoming effective during qubit initialization and readout:

In [ ]:
my_circuit.apply_initialization_noise(Noise.GeneralizedAmplitudeDamping(probability=0.05, gamma=0.1))

my_circuit.apply_readout_noise(Noise.PhaseFlip(probability=0.1))

print(my_circuit)

And finally, we can a noise channels bottom-up like a logic gate acting on a specific qubit at a specific position in the sequence of gates:

In [ ]:
my_circuit.pauli_channel(2, probX=0.1, probY=0.2, probZ=0.3)
print(my_circuit)

Now, let's simulate this circuit:

In [ ]:
task = device.run(my_circuit, shots=1000)

# visualize the results
result = task.result()
measurement = result.measurement_counts
print(measurement)

plt.bar(measurement.keys(), measurement.values())
plt.xlabel('bitstrings')
plt.ylabel('counts')
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.draw()